# IEOR 242 Project. Predicting Box Office <a class="tocSkip">

The goal of this file is to preprocess the original datasets collected from https://www.imdb.com/interfaces/ to create a new dataset on top of which several statistical learning models will be applied to try to predict Box Office. 

#### Import libraries <a class="tocSkip">

In [1]:
# General libraries
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns

#### General title information datasets <a class="tocSkip">

In [3]:
# General information about the titles
title_akas = pd.read_csv("title_akas.tsv", sep='\t')
title_basics = pd.read_csv("title_basics.tsv", sep='\t')

C:\Users\esthe\anaconda3\envs\242\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\esthe\anaconda3\envs\242\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


The filters applied to the original datasets are the following:
* Only movies will be considered. No TV series or short movies will be examined.
* Among the set of these movies, only those corresponding to the US released after 1990 will be included in the final dataset. 


In [4]:
# Filter movies by region. Only those corresponding to the US will be selected
us_movies = title_akas[title_akas['region']=='US']

In [5]:
# ID of the US movies
usMoviesID=us_movies['titleId']

In [6]:
# Select only those movies in the second file corresponding to US movies
titleBasicsUS = title_basics[title_basics['tconst'].isin(usMoviesID)]

In [7]:
# Only movies will be considered. No TV series or short movies
longUsMovies = titleBasicsUS[titleBasicsUS['titleType']=='movie']
df = longUsMovies['startYear'].apply(pd.to_numeric, errors='coerce')
longUsMovies = longUsMovies.assign(startYear = df)
LongUsMoviesWoNa = longUsMovies.dropna(axis = 0, subset = ['startYear'])

In [42]:
# Filter movies by year of release after 1990
movieSelection = LongUsMoviesWoNa[LongUsMoviesWoNa['startYear'] > 1990]
print ("The total number of US movies after 1990 is :", len(movieSelection))

The total number of US movies after 1990 is : 147279


In [43]:
movieSelection.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34805,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,\N,118,"Comedy,Fantasy,Romance"
35545,tt0036177,movie,Muhomatsu no issho,Muhomatsu no issho,0,2008.0,\N,100,"Action,Adventure"
61123,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,\N,70,Drama
64097,tt0065392,movie,Bucharest Memories,Amintiri bucurestene,0,2020.0,\N,\N,Documentary
67676,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,\N,122,Drama


Given the dataset of US movies, let's check which columns are meaningless and hence could be dropped.

In [44]:
us_movies.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
35,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
39,tt0000005,5,Blacksmith Scene #1,US,\N,alternative,\N,0
40,tt0000005,6,Blacksmithing,US,\N,\N,informal alternative title,0


In [45]:
# Exploration of column types
us_movies.groupby('types').sum()

,ordering
types,
\N,523986
alternative,74164
dvd,12718
dvdvideo,7
festival,4449
festivalworking,7
imdbDisplay,2297766
original,47
tv,6675


Regarding the column type, the bast majority of movies either correspond to imdbDisplay or have a missing value. The column types will not be considered as it gives no value at all. 

In [46]:
# Exploration of column languages
us_movies.groupby('language').sum()

,ordering
language,
\N,3075466
cr,3
en,19811
es,4399
fr,2
haw,16
hi,4
myv,3
yi,241


Regarding the language, given that only movies which region corresponds to the US have been selected, the bast majority of them are either english speaking movies of have no value. The column language will not be considered as it gives no value at all. 

In [47]:
# Exploration of column attributes
grouped = us_movies.groupby('attributes').sum()
grouped.sort_values('ordering', ascending=False)

,ordering
attributes,
\N,2888388
complete title,18832
alternative spelling,17288
reissue title,13441
new title,11445
...,...
YIVO translationreissue title,2
dubbed versionrecut version,2
original pilot title,2


Regarding the column attributes, most of movies have a missing value, and the rest of possibilities are not very descriptive either. Hence, this column will not be considered either.

Given the dataset of movies after 1990, let's check which columns are meaningless and hence could be dropped. 

In [48]:
# endYear column
movieSelection.groupby('endYear').count() 

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
endYear,,,,,,,,
\N,147279,147279,147279,147279,147279,147279,147279,147279


The endYear is a NaN value for all movies, as it only made sense for TV episodes. Hence, this column will be removed. 

In [49]:
# titleType column
movieSelection.groupby('titleType').count() 

,tconst,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
titleType,,,,,,,,
movie,147279,147279,147279,147279,147279,147279,147279,147279


The titleType column is the same for all movies, as it only made sense when there were titles that corresponding both to short movies, TV series and long movies. Hence, this column will be removed. 

In [50]:
movies_selection_filtered = movieSelection.drop(columns=["endYear", "titleType"])
movies_selection_filtered.head()

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34805,tt0035423,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
35545,tt0036177,Muhomatsu no issho,Muhomatsu no issho,0,2008.0,100,"Action,Adventure"
61123,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,70,Drama
64097,tt0065392,Bucharest Memories,Amintiri bucurestene,0,2020.0,\N,Documentary
67676,tt0069049,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama


Let's create several categorical variables for the column genre to use.

In [51]:
# Convert the genres column into a list of strings
movies_selection_filtered['genres'] = movies_selection_filtered['genres'].apply(lambda x: x.split(','))

In [52]:
# Convert the list of strings into categorical variables
movies_selection_filtered_dummies = pd.get_dummies(movies_selection_filtered.genres.apply(pd.Series).stack()).sum(level=0)

C:\Users\esthe\AppData\Local\Temp/ipykernel_25604/3926321169.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  movies_selection_filtered_dummies = pd.get_dummies(movies_selection_filtered.genres.apply(pd.Series).stack()).sum(level=0)


In [53]:
# Only those categorical variables that appear in more than 5000 movies will be selected
movies_selection_filtered_dummies.sum() < 5000

Action         False
Adult           True
Adventure      False
Animation       True
Biography      False
Comedy         False
Crime          False
Documentary    False
Drama          False
Family         False
Fantasy         True
Game-Show       True
History         True
Horror         False
Music           True
Musical         True
Mystery        False
News            True
Reality-TV      True
Romance        False
Sci-Fi          True
Short           True
Sport           True
Talk-Show       True
Thriller       False
War             True
Western         True
\N              True
dtype: bool

In [54]:
movies_filtered_5000 = movies_selection_filtered_dummies.loc[:, movies_selection_filtered_dummies.sum() < 5000]
movies_filtered_5000 = movies_filtered_5000.drop(columns=[r'\N'])
movies_filtered_5000

,Adult,Animation,Fantasy,Game-Show,History,Music,Musical,News,Reality-TV,Sci-Fi,Short,Sport,Talk-Show,War,Western
34805,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
35545,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
61123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
64097,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
67676,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8462828,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8462968,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8462977,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8463061,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [55]:
movies_filtered_5000

,Adult,Animation,Fantasy,Game-Show,History,Music,Musical,News,Reality-TV,Sci-Fi,Short,Sport,Talk-Show,War,Western
34805,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
35545,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
61123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
64097,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
67676,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8462828,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8462968,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8462977,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8463061,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [56]:
movies_selection_filtered

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
34805,tt0035423,Kate & Leopold,Kate & Leopold,0,2001.0,118,"[Comedy, Fantasy, Romance]"
35545,tt0036177,Muhomatsu no issho,Muhomatsu no issho,0,2008.0,100,"[Action, Adventure]"
61123,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,70,[Drama]
64097,tt0065392,Bucharest Memories,Amintiri bucurestene,0,2020.0,\N,[Documentary]
67676,tt0069049,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,[Drama]
...,...,...,...,...,...,...,...
8462828,tt9915872,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,97,"[Comedy, Drama, Fantasy]"
8462968,tt9916170,The Rehearsal,O Ensaio,0,2019.0,51,[Drama]
8462977,tt9916190,Safeguard,Safeguard,0,2020.0,90,"[Action, Adventure, Thriller]"
8463061,tt9916362,Coven,Akelarre,0,2020.0,92,"[Drama, History]"


In [59]:
# Merge genre categorical variables with original dataset
movies_selection_genre = movies_selection_filtered.join(movies_filtered_5000)
movies_selection_genre = movies_selection_genre.drop(columns = ['genres'])
movies_selection_genre.head()

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,Adult,Animation,Fantasy,Game-Show,...,Music,Musical,News,Reality-TV,Sci-Fi,Short,Sport,Talk-Show,War,Western
34805,tt0035423,Kate & Leopold,Kate & Leopold,0,2001.0,118,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
35545,tt0036177,Muhomatsu no issho,Muhomatsu no issho,0,2008.0,100,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61123,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,70,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64097,tt0065392,Bucharest Memories,Amintiri bucurestene,0,2020.0,\N,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67676,tt0069049,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
# Save combined file
movies_selection_genre.to_csv("movies_selection_categorical.csv", index=False)

#### Rating information datasets <a class="tocSkip">

In [62]:
# Read title ratings files
title_ratings = pd.read_csv("title_ratings.tsv", sep='\t')

In [63]:
# Select those columns in movieSelection
title_ratings = title_ratings[title_ratings['tconst'].isin(movieSelection['tconst'])]
len(title_ratings)

99150

In [64]:
title_ratings.head()

,tconst,averageRating,numVotes
17794,tt0035423,6.4,81936
18427,tt0036177,7.3,113
40529,tt0062336,6.3,157
43167,tt0065392,7.4,12
46361,tt0069049,6.8,6930


#### Datasets merging <a class="tocSkip">

In [65]:
# Merge genral title info with title ratings
merged_df = movies_selection_genre.join(title_ratings.set_index('tconst'), on='tconst')

In [66]:
# Inspect columns of merged dataset
merged_df.columns

Index(['tconst', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'Adult', 'Animation', 'Fantasy', 'Game-Show',
       'History', 'Music', 'Musical', 'News', 'Reality-TV', 'Sci-Fi', 'Short',
       'Sport', 'Talk-Show', 'War', 'Western', 'averageRating', 'numVotes'],
      dtype='object')